# Assignment 2A

In [1]:
# Importing Necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

In [2]:
# Importing Data
returns = pd.read_excel('Data_2A_2023.xlsx',index_col='Date') / 100
returns.describe()

,AAPL,DIS,GE,GS,MSFT
count,5787.000000,5787.000000,5787.000000,5787.000000,5787.000000
mean,0.001213,0.000422,0.000059,0.000540,0.000513
std,0.025148,0.019490,0.021196,0.023153,0.019388
min,-0.518691,-0.183630,-0.151592,-0.189596,-0.155978
25%,-0.010370,-0.008772,-0.008820,-0.010168,-0.008268
50%,0.000872,0.000289,0.000000,0.000307,0.000347
75%,0.013145,0.009135,0.008968,0.011173,0.009376
max,0.139050,0.159722,0.197031,0.264678,0.195652


In [3]:
# Train, Test Split
train_set = returns[:"2015"]
test_set = returns["2016":]

# Estimate parameters
average_return = train_set.mean()
covariance_matrix = train_set.cov(ddof=0)

In [4]:
def optimize_portfolio(expected_returns, cov_matrix, required_return):
    """
    Optimizes the vector of portfolio weights according to the first optimization problem in Assignment 2A
    """
    # Define Decision variables
    num_assets = len(expected_returns)
    weights = cp.Variable(num_assets, nonneg=False)
    
    # Compute Performance Metrics
    portfolio_return = cp.matmul(weights, expected_returns)
    portfolio_variance = cp.quad_form(weights, cov_matrix)
    
    # Set up formulation
    constraints = [cp.sum(weights) == 1, portfolio_return == required_return]
    objective = cp.Minimize(portfolio_variance)
    optimization_problem = cp.Problem(objective, constraints)
    
    # Solve Optimization Problem
    optimization_problem.solve()
    
    # Return Solution
    if optimization_problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        return weights.value
    else:
        print('No solution found')
        return None 

In [5]:
average_positive_return = average_return[average_return > 0].mean()
optimized_weights = optimize_portfolio(average_return, covariance_matrix, average_positive_return)
print(optimized_weights)

[0.1960708  0.33860056 0.19563236 0.03650267 0.23319361]


In [6]:
def calculate_performance(portfolio_weights, average_returns, covariance_matrix):
    """ 
    Calculate the annualised performance metrics of the portfolio based on weights and mean and cov estimates.
    """
    annual_portfolio_return = np.dot(portfolio_weights, average_returns) * 252
    annual_portfolio_variance = np.dot(portfolio_weights, np.dot(covariance_matrix, portfolio_weights)) * 252
    return annual_portfolio_return, annual_portfolio_variance

def compute_portfolio_returns(returns_df, optimized_weights):
    """
    Computes the portfolio returns over time, accounting for drift in weights over time.
    """
    current_weights = optimized_weights.copy()
    dates = returns_df.index
    portfolio_returns = []  

    for current_date in dates:
        # Calculate the return for the current date by dot product of returns and current weights
        portfolio_returns.append(np.dot(returns_df.loc[current_date], current_weights))
        
        # Recompute drifted weights based on days returns
        drifted_weights = current_weights * (1 + returns_df.loc[current_date])
        total_value = np.sum(drifted_weights)
        current_weights = drifted_weights / total_value
        
    portfolio_returns_df = pd.Series(portfolio_returns, index=dates)
    
    return portfolio_returns_df

In [7]:
# Calculating performance of the optimized portfolio
annual_return, annual_variance = calculate_performance(optimized_weights, average_return, covariance_matrix)
print(f'Predicted annual return: {annual_return:.2%}')
print(f'Predicted annual variance: {annual_variance:.3f}')

# Analyzing performance on the testing set
mv_portfolio_returns = compute_portfolio_returns(test_set, optimized_weights)
print(f'Annualised Portfolio Mean Return: {mv_portfolio_returns.mean() * 252:.2%}')
print(f'Annualised Portfolio Variance: {mv_portfolio_returns.var() * 252:.3f}')

Predicted annual return: 14.03%
Predicted annual variance: 0.066
Annualised Portfolio Mean Return: 16.82%
Annualised Portfolio Variance: 0.059


In [8]:
def minimize_variance(covariance_matrix):
    """
    Optimizes the vector of portfolio weights according to the second optimization problem in Assignment 2A
    """
    # Define Decision Variables
    num_assets= covariance_matrix.shape[0]
    weights = cp.Variable(num_assets, nonneg=False)
    
    # Define Performance Metrics
    portfolio_variance = cp.quad_form(weights, covariance_matrix)
    
    # Define Formulation
    constraints = [cp.sum(weights) == 1]
    objective = cp.Minimize(portfolio_variance)
    problem = cp.Problem(objective, constraints)
    
    # Solve Optimization Problem
    problem.solve()
    
    # Return solution
    if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        return weights.value
    else:
        print('No solution found')
        return None 
    
minimum_variance_weights = minimize_variance(covariance_matrix)
print(minimum_variance_weights)

[0.09035822 0.28880301 0.29381745 0.02033789 0.30668343]


In [11]:
# Calculating performance of the minimum variance portfolio on the trainingset
annual_return, annual_variance = calculate_performance(minimum_variance_weights, average_return, covariance_matrix)
print(f'Predicted annual return: {annual_return:.2%}')
print(f'Predicted annual variance: {annual_variance:.3f}')

# Analyzing performance on the testing set
min_var_portfolio_returns = compute_portfolio_returns(test_set, optimized_weights)
print(f'Annualised Portfolio Mean Return: {min_var_portfolio_returns.mean() * 252:.2%}')
print(f'Annualised Portfolio Variance: {min_var_portfolio_returns.var() * 252:.3f}')

Predicted annual return: 10.79%
Predicted annual variance: 0.064
Annualised Portfolio Mean Return: 16.82%
Annualised Portfolio Variance: 0.059
